# <font color='#8735fb'> **Airline Delays - ML Workflow** </font> 
> <font color='#8735fb'> [ single CPU/ GPU ] </font>

RAPIDS aims to contribute to data science by building GPU accelerated versions of open-source data science libraries. 

With RAPIDS and GPUs the inherent parallelism of the data science workflow is exposed to massive compute making possible a qualitative improvement in the life of a practicing data scientist, as well as substantial speedups for large scale machine learning in production.

<img src='images/airline_dataset.png' width='1250px'>

<div style="text-align: right"> 
<a href='https://d7vw40z4bofef.cloudfront.net/static/2.37.07-web19/images/service/isometric/flight.svg'>image source</a> </div>

In [ ]:
import numpy, pandas
import sklearn, xgboost        # python data science stack
import cupy, cudf, cuml        # RAPIDS

In [ ]:
import teach_ML, benchmark    # helper functions, benchmaking

In [ ]:
import importlib               # dynamic code reloading
importlib.reload( teach_ML ); 
importlib.reload( benchmark );

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display
import ipywidgets

import warnings; warnings.filterwarnings('ignore')

## <font color='#8735fb'> Motivation </font>

In this notebook we'll train a machine learning model to anticipate flight delays -- i.e., to determine if a flight will arrive more than 15 minutes past its scheduled time. Accurately predicing late flights is an application of machine learning that could be used to help improve airport operation, carrier logistics, and/or consumer travel planning. 

As we go forward with the workflow we'll narrate what's happening with the data as it's prepared for ingestion by our model. Along the way we'll show the steps we take with code written using the data science stack of `pandas`, `numpy`, and `sklearn` as well as code using their RAPIDS euqivalents `cupy`, `cudf`, and `cuml`.

Workflow Steps:

> **1. Performance Tracking**

> **2. Data Ingestion**

> **3. ETL**
-> handle missing -> encode non-numerics -> split

> **3. Explore**
-> cross-correlation 
-> visualization [ TODO: dynamic plot ] 
-> finalize feature selection 

> **4. Train Classifier**
-> XGBoost vs RF

> **5. Inference**
-> FIL

> **6. Advanced Topics / Extensions [ optional ]**
-> interactive multi-plot viz, onramp to DL

### <font color='#8735fb'> **Dataset** </font>

At the heart of our analysis will be domestic carrier on-time reporting data that has been kept for decades by the U.S. Bureau of Transportation.

This rich source of data allows us to scale, so while in this notebook (ML_100.ipynb) we only use 1 GPU and 1 year of data, in the next notebook (ML200.ipynb) we'll use 10 years of data and multiple GPUs.

> **Dataset**: [US.DoT - Reporting Carrier On-Time Performance, 1987-Present](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236)

The public dataset contains logs/features about flights in the United States (17 airlines) including:

* locations and distance  ( `Origin`, `Dest`, `Distance` )
* airline / carrier ( `Reporting_Airline` )
* scheduled departure and arrival times ( `CRSDepTime` and `CRSArrTime` )
* actual departure and arrival times ( `DpTime` and `ArrTime` )
* difference between scheduled & actual times ( `ArrDelay` and `DepDelay` )
* binary encoded version of late, aka our target variable ( `ArrDelay15` )

> Note: We limit this first workflow to a single calendar year (Jan - Dec of 2019) which is about 7.5M flights. As an extension we also look at how you might train a model ensemble to handle more recent data (see [impacts of COVID-19](https://www.bts.gov/data-spotlight/march-day-day-how-flight-cancellations-rose-17)). 

## <font color='#8735fb'> 1. Performance Tracking </font>

Whenever we run a piece of our pipeline we'll be able to benchmark its performance on the compute type of our choice and update the value in a log dictionary. Let's start by initializing an empty dictionary.

In [ ]:
log = {}

#### <font color='#8735fb'> Compute Choice </font>

We have the option of selecting to run either only-GPU or both CPU and GPU code cells. Use the widget below to make your selection.

In [ ]:
compute_choice = ipywidgets.ToggleButtons( description = 'compute:', options = ['GPU', 'GPU & CPU'] )

display( compute_choice )

#### <font color='#8735fb'> Benchmarking Context </font>

We'll wrap our code in a context manager object that allows us to 
* gracefully handle error catching 
* updates our result `log` with the duration of the code body
* allows us to specify the compute requirements of the execution block [ and match the user's `compute_choice` ]

For more on how the python with statement and context managers work see [this great writeup](https://effbot.org/zone/python-with-statement.htm), or [the python docs](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement), and/or check out the code in `benchmark.py`

An example of how we use the benchmarking context manager is shown below.

```python
In[0]: with benchmark.GPU ( log, 'ingestion', compute_choice ) as GPU_context:
  ...:     if GPU_context.execute_block == True:
  ...:         # execution body
  ...:         data = cudf.read_csv( data_dir + airline_stats, index_col = 0)
```    

## <font color='#8735fb'> 2. Data Ingestion </font> 

### <font color='#8735fb'> Loading Airline Dataset [2019] <font>
> customize for your own dataset

In [ ]:
data_dir = '/workspace/ml/data/'
airline_stats = '2019_airlines.csv'

In [ ]:
csv_filename = str(data_dir + airline_stats)

In [ ]:
data = cudf.DataFrame()
data_cpu = pandas.DataFrame()

###  <font color='#8735fb'> 2.1. CPU Ingestion </font>

In [ ]:
with benchmark.CPU ( log, 'ingestion', compute_choice ) as CPU_context:
    if CPU_context.execute_block == True:
        
        data_cpu = pandas.read_csv( csv_filename, index_col = 0)

### <font color='#8735fb'> 2.2. GPU Ingestion </font> 

In [ ]:
with benchmark.GPU ( log, 'ingestion', compute_choice ) as GPU_context:
    if GPU_context.execute_block == True:
        
        data = cudf.read_csv( csv_filename, index_col = 0)

In [ ]:
teach_ML.compare_speedups(log);

### <font color='#8735fb'> 2.3. Validation </font>

Let's now take a look at what has been loaded into our dataframe.

In [ ]:
print( f'data shape on GPU : {data.shape} \n'
       f'data shape on CPU : {data_cpu.shape}') 

As the shape of our data tells us, this is a classic tabular dataset -- i.e., very tall and narrow (almost 300,000 times longer than it is wide).

In [ ]:
data.head().to_pandas().T

In [ ]:
data_cpu=data_cpu.dropna()

In [ ]:
data=data.dropna()

### <font color='#8735fb'>[ Optional ] Augmentation</font>

Modern data scientists have an ever growing arsenal of public datasets that can be combined with their base data. In our workflow, we'll add some additional information by adding in airport locations and the human readable airline carrier names (rather than their coded versions). These additions will help us in upstream interpretation and visualization. 

More spefically we'll merge in
> full-string names of airlines matched on the carrier codes in the baseline dataset [`carriers.csv`]

> airport locations (lat,lng) matches on airport codes in the baseline dataset [`airports.csv`]

In [ ]:
augment_choice = ipywidgets.ToggleButtons( description = 'augment:', options = ['Yes', 'No'] )

display( augment_choice )

In [ ]:
if augment_choice.value == 'Yes':    
    with benchmark.CPU ( log, 'augmentation', compute_choice ) as run_context:        
        if run_context.execute_block == True:
            
            data_cpu = teach_ML.augment_dataset_inplace ( data_cpu, data_dir, pandas )        

In [ ]:
if augment_choice.value == 'Yes':
    with  benchmark.GPU ( log, 'augmentation', compute_choice ) as run_context:
        if augment_choice.value == 'Yes' and run_context.execute_block == True:
            
            data = teach_ML.augment_dataset_inplace  ( data, data_dir, cudf )

# <font color='#8735fb'> 3. ETL </font>
    

Now that we have our dataset loaded, our goal is to prepare it for analysis and eventual ingestion by a Machine Learning model.

To this end we need to come up with a strategy for handling missing data and for encoding non-numeric data.

> !BEWARE: 
* several of the code cells below delete and/or modify the dataset
* running them more than once and/or out of order may break the expected downstream logic
* its always possible to return to the ingestion stage to start with a fresh dataset

### <font color='#8735fb'> 3.1. Handle Missing Data </font>

Only a very small fraction of flights have missing data (about 0.021% in 2019).

It turns (not surprisingly in retrospect) that the dataset entries with missing values are almost exclusively instances of flight cancellations.

For those who are curious we invite you to explore further (e.g, ideas to get you started):
```python
data['Cancelled'].value_counts()
data.dropna()['Cancelled'].value_counts()
```

Since cancelled flights are not relevant for our upstream prediction of arrival delays (i.e., cancelled flights can be labeled as being late without training a model), we feel comfortable dropping all data elements with missing values.

In [ ]:
with benchmark.CPU ( log, 'ETL.dropna', compute_choice ) as run_context:        
    if run_context.execute_block == True:
        
        data_cpu = data_cpu.dropna() # inplace drop samples w/ missing values

In [ ]:
with benchmark.GPU ( log, 'ETL.dropna', compute_choice ) as run_context:        
    if run_context.execute_block == True:
        
        data = data.dropna() # inplace drop samples w/ missing values

In [ ]:
teach_ML.compare_speedups(log); # only for the most recent

### <font color='#8735fb'> 3.2. Handle Non-Numeric Data </font>

Note that some variables are numeric and easily lend themselves to being packaged into inputs for upstream modelling. 

Others (e.g., Origin, Dest, DestCityName, etc) are strings/non-numeric and will require a bit more effort to be included. 

We can enumerate the possible values of non-numeric features, and re-map them to integers using a category datatype conversion `.astype('category')`. To preseve the original data, we'll add/append the newly encoded version of the categorical features into columns with the `enc_` prefix.

In [ ]:
encodings, mappings = data['OriginCityName'].factorize() # encode/categorize a sample feature 

In [ ]:
list( zip ( data['OriginCityName'][0:10].values_host, encodings[0:10] ) )

###  <font color='#8735fb'> 3.2.1 Encode and append </font>

In [ ]:
numeric_columns = []

In [ ]:
data.columns

In [ ]:
with benchmark.CPU ( log, 'ETL.encode', compute_choice ) as run_context:
    if run_context.execute_block == True:
        for colname in data_cpu.columns:
            if data_cpu[colname].dtype == object:

                values = data_cpu[colname].astype('category').cat.codes.astype('float32') # encode
                colname = 'enc_' + colname
                
                data_cpu.insert( len(data_cpu.columns), colname, values ) # add encoded column

In [ ]:
with benchmark.GPU ( log, 'ETL.encode', compute_choice ) as run_context:
    if run_context.execute_block == True:
        for colname in data.columns:
            if data[colname].dtype == object:
                
                values = data[colname].astype('category').cat.codes.astype('float32')
                colname = 'enc_' + colname                
                data.insert(0, colname, values)                
                
            numeric_columns += [ colname ]  

### <font color='#8735fb'> 3.3. Split Dataset into Train and Test </font>

#### <font color='#8735fb'> 3.3.1. Filter Input Features </font>

In [ ]:
target = set ( ['ArrDel15'] ); 
target_surrogates = set ( ['ArrTime', 'ArrDelay'])

redundant_cols = set( [ 'Year', 'Cancelled', 'DOT_ID_Reporting_Airline', 'enc_IATA_CODE_Reporting_Airline' ] )

target_column = list ( target  )
input_columns = list ( set( numeric_columns )\
                            .difference( target )\
                            .difference( target_surrogates )\
                            .difference( redundant_cols) )
input_columns.sort()

#### <font color='#8735fb'> 3.3.2. Split (80%, 20%) <font>

In [ ]:
with benchmark.CPU ( log, 'ETL.split', compute_choice ) as run_context:
    if run_context.execute_block == True:
        
        from sklearn.model_selection import train_test_split

        X_train_cpu, X_test_cpu, \
            y_train_cpu, y_test_cpu = train_test_split( data_cpu[input_columns], 
                                                        data_cpu[target_column],
                                                        train_size = 0.80, 
                                                        random_state = 42 )

In [ ]:
with benchmark.GPU ( log, 'ETL.split', compute_choice ) as run_context:
    if run_context.execute_block == True:

        train_test_split = cuml.preprocessing.model_selection.train_test_split

        X_train, X_test, \
            y_train, y_test = train_test_split( X = data[input_columns],  
                                                y = data[target_column], 
                                                train_size = 0.80, 
                                                random_state = 42 )

## <font color='#8735fb'> 4. Explore </font>

Let's review the current state of our training data. We should have a dataframe with no missing values, and all numerical encoded features.

In [ ]:
X_test.head(5).to_pandas().round(2).T

### <font color='#8735fb'> 4.1. Cross-correlation </font>

In data science correlation matrix is frequently used to explain the strength of linear relationship between variables. Understanding the linear relationship between variables is useful because, if such relationship is existing, we can use the value of one variable to predict the value of the other variable.

Computing cross-correlation in a large dataset is computationally expensive and yet easy to accelerate on GPU. 
<br>Here we can take advantage of [`CuPy`](https://cupy.chainer.org/) library which is GPU implementation of general-purpose array-processing library NumPy. 

In [ ]:
correlation_columns = input_columns + target_column
correlation_columns.sort()

In [ ]:
with benchmark.CPU ( log, 'explore.corrcoef', compute_choice ) as run_context:
    if run_context.execute_block == True:

         cov_cpu = numpy.corrcoef( data_cpu[ correlation_columns ].values, rowvar = False)

In [ ]:
with benchmark.GPU ( log, 'explore.corrcoef', compute_choice ) as run_context:
    if run_context.execute_block == True:
        
        cov = cupy.corrcoef( data[ correlation_columns ].as_gpu_matrix(), rowvar = False)

In [ ]:
ax = sns.heatmap( data = numpy.round( cupy.asnumpy(cov), 2),
                  annot = True, linewidth = .5, 
                  cmap = sns.diverging_palette( 150, 275, s = 80 ),
                  xticklabels = correlation_columns,
                  yticklabels = correlation_columns,
                  figure = plt.figure(figsize=(15,10)) )

Take some time to observe the plot above.

Values that you see in the cells are correlation coefficients that indicate how strong the relationship is. It ranges between -1 and 1. Closer it is to those extreme values, stronger the linear relationship between 2 variables is. 
You should notice that cells with higher numbers are having more intense color. This just makes it easier to find variables with stronger positive (purple) or negative (green) relationship. 

For your reference, you can see a clarification on the variables we used [here](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236) 

You can note a few important observations from the plot above: 
1. Observe the variables that are highly corelated to our variable of interest: *ArrDel15*.<br/>
You might have noticed that this binary variable has stronger linear relationship with other variables that indicate Arrival/Departure time and delay 
2. Perfect correlation between *ArrDelay* and *DepDelay* and strong relationship (0.7) between binary indicators *ArrDel15* and *DepDel15*.<br/>
This tells us that if we know that one flight is about to depart earlier or later than scheduled, we can use this as a very strong indicator for arrival delay.
High correlation coefficient between *ArrDel15* and *DepDel15* tells us that delayed flights are very likely to arrive to their destination late. 
3. There are many variables in this dataset that are not linearly related to each other. However, later in this tutorial we will show that advanced ML algorithms can still deduce useful and actionable information from this data even though simple ML Algorithms such as linear regression might fare poorly against lack of linear relationships.


If you are interested to look into the code we used to generate the cross-correlation, you can uncomment and execute the cell below. 

### <font color='#8735fb'> 4.2. Filter & Visualize </font>

In [ ]:
data['OriginCityName']

In [ ]:
filtered = data [ data['OriginCityName'] == 'Seattle, WA' ]
filtered['height'] = filtered['ArrDel15'] * 1 + filtered['ArrDelay'].scale()

In [ ]:
renderer = teach_ML.geo_plot( filtered )
renderer.to_html( filename='geo.html', iframe_height = 500, iframe_width = 1200)

In [ ]:
filtered = data [ data['OriginCityName'] == 'Atlanta, GA' ]
filtered['height'] = filtered['ArrDel15'] * 1 + filtered['ArrDelay'].scale()

In [ ]:
renderer = teach_ML.geo_plot( filtered )
renderer.to_html( filename='geo.html', iframe_height = 500, iframe_width = 1200)

## <font color='#8735fb'> 5. Model </font>

Flight delays are causing not just an inconvenience to passengers, but millions of dollars in damage to airlines and supporting businesses. 
Accurate prediction of flight delays can greatly reduce the economic loss caused by said delays. As an added benefit, travelers can get timely insights into potential issues with their future travel.<br/>

We will use a year's worth of data to look for common patterns in late arrivals. These include factors like location, distance, airlines and whether the aircraft is departing late. 
We will use Random Forest to predict whether a flight will arrive on time


If you are curious, here is a helpful figure visualizing <a href='images/decision_tree_building.png'>how decisions trees are built</a>.

<font color='#8735fb'> **Set Parameters**  </font>

In [ ]:
model_params_rf = {
    'n_estimators':10,
    'max_depth':5
}

# <font color='#8735fb'> **GPU Model Training and Inference** </font>

In [ ]:
rf_trained_model_cpu = None
rf_trained_model = None

In [ ]:
%%time
with benchmark.GPU (log, 'rf.train', compute_choice ) as run_context:
    if run_context.execute_block == True:
        
        from cuml.ensemble import RandomForestClassifier
        
        rf_model = RandomForestClassifier(
            n_estimators=model_params_rf['n_estimators'],
            max_depth=model_params_rf['max_depth'],
        )
        trained_model = rf_model.fit(
            X_train.astype('float32'), 
            y_train.astype('int32')
        )

In [ ]:
with benchmark.GPU (log, 'rf.inference', compute_choice ) as run_context:
    if run_context.execute_block == True:
        
        predictions = trained_model.predict(
            X_test.astype('float32')
        )

# <font color='#8735fb'> **CPU Model Training and Inference** </font>

In [ ]:
%%time 
with benchmark.CPU (log, 'rf.train', compute_choice ) as run_context:
    if run_context.execute_block == True:
        
        from sklearn.ensemble import RandomForestClassifier
        
        rf_model_cpu = RandomForestClassifier(
            n_estimators=model_params_rf['n_estimators'],
            max_depth=model_params_rf['max_depth'],
            n_jobs=-1
        )
        trained_model_cpu = rf_model_cpu.fit(
            X_train_cpu.astype('float32'),
            y_train_cpu.astype('int32')
        )
        

In [ ]:
with benchmark.CPU (log, 'rf.inference', compute_choice ) as run_context:
    if run_context.execute_block == True:
        
        predictions_cpu = trained_model_cpu.predict(
            X_test_cpu.astype('float32')
        )
        

## <font color='#8735fb'> Review Perf </font>


In [ ]:
matched_ops = teach_ML.compare_speedups( log )

In [ ]:
teach_ML.polar_plot_results( matched_ops, False )

# <font color='#8735fb'> References </font>

> [Airline Dataset](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236) 

> [RAPIDS website ](https://rapids.ai/) 

> [Forest Inference Library ](https://medium.com/rapids-ai/rapids-forest-inference-library-prediction-at-100-million-rows-per-second-19558890bc35)

> [XGboost](https://xgboost.readthedocs.io/en/latest/)
